In [ ]:
# usual imports
import os
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params

In [ ]:
bands = 'grizy'
maglims = [27.66, 27.25, 26.6, 26.24, 25.35]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"HSC{band}_cmodel_dered"] = limx

In [ ]:
common_params.set_param_defaults(
    bands=[f'HSC{band}_cmodel_dered' for band in bands],
    err_bands=[f'{band}_cmodel_magerr' for band in bands],
    nondetect_val=np.nan,
    ref_band='HSCi_cmodel_dered',
    redshift_col='specz_redshift',
    mag_limits=maglim_dict,
    zmax = 6.0,
)

In [ ]:
pipe = InformPipeline()

In [ ]:
dir(pipe)

In [ ]:
pipe.inform_knn.config.update(leaf_size=14)

In [ ]:
pipe.inform_knn.config

In [ ]:
pipe.print_stages()

In [ ]:
datadir = '/Users/jtm/data/lsst/PZ/'
input_file = os.path.join(datadir, 'dered_223501_sz_match_pdr3_dud_NONDET.hdf5') # hsc_train/

input_dict = dict(
    input=input_file,
)

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=True), None)

In [ ]:
pipe.save('tmp_inform_all.yml')

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_inform_all.yml')

In [ ]:
pr.run()